# NBA Data analysis

The data set used in this exercise is taken from https://www.kaggle.com/nathanlauga/nba-games

## Imports section

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

## Data set loading

Note: loading the following CSV will give a DtypeWarning because columns (6) have mixed types. This quite trivial information could be quite important in the following steps

In [ ]:
path = os.path.join(os.path.curdir,"sample_data", "games_details.csv")
df = pd.read_csv(path)

df

## Exploratory Data Analysis

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe().T

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.tail()

## Null values management and data wrangling

In [ ]:
df1 = df.dropna()
df1

This simple operation (done in a separate dataframe to maintain the integrity of original loaded data) show us that all rows have at least a missing value

In [ ]:
df['COMMENT'].isnull().sum()

### Exercise 

Filter data frame to obtain only rows with null comment

In [ ]:
has_null_comments = df.COMMENT.isnull()
df[has_null_comments]

### Exercise

Drop the columns that end with "_ID", possibly in one line

In [ ]:
df = df.drop([col for col in df.columns if col.endswith("_ID")], axis=1)
df

### Exercise

Find null values in MIN

In [ ]:
has_null_min = df.MIN.isnull()
df[has_null_min]

## Plotting

### Exercise

Plot a bar chart with the top 10 players that played the most matches

In [ ]:
# Counts how may rows contains the player name
# Each row represents a played game
most_played_df = df.PLAYER_NAME.value_counts(ascending=False)
# Keep only the top 10, our target
most_played_df = most_played_df[0:10]

In [ ]:
plt.figure(figsize=(10, 10))
plt.bar(most_played_df.index, most_played_df)
plt.xticks(most_played_df.index, rotation=45)
plt.title('Top 10 players by games played')
plt.ylabel('Games played')
plt.show()

## Statistics extraction

### Exercise

Find player that has the highest PLUS_MINUS on average (mean)

In [ ]:
# Check if the affected column has null values
df['PLUS_MINUS'].isnull().sum()

In [ ]:
# Keep only the subset of columns I need to calculate the required result
columns_to_keep = ['PLAYER_ID', 'PLAYER_NAME', 'PLUS_MINUS']
plus_minus_df = df.drop([col for col in df.columns if col not in columns_to_keep ], axis=1)

In [ ]:
# Remove null values
plus_minus_df_valued_only = plus_minus_df.dropna()

In [ ]:
# Group by player name summing PLUS_MINUS
plus_minus_df_grouped = plus_minus_df_valued_only.groupby('PLAYER_NAME').agg({'PLUS_MINUS': 'mean'})

# Find the highest value index
highest_value_idx = plus_minus_df_grouped.PLUS_MINUS.argmax()
plus_minus_df_grouped.iloc[highest_value_idx]

#### Solution (if there are no mistakes): Quenton Jackson

### Exercise

Find player that has the most AST (assists)

In [ ]:
# Pseudocode - Step by step guide
# 1. Keep only subset of columns of interest
# 2. Drop null values
# 3. Group by player
# 4. Sum AST column values
# 5. Keep the highest one

In [ ]:
columns_to_keep_ex_2 = ['PLAYER_ID', 'PLAYER_NAME', 'AST']
ast_df = df.drop([col for col in df.columns if col not in columns_to_keep_ex_2], axis=1)

In [ ]:
ast_df.AST.isnull().sum()

In [ ]:
ast_df.dropna()

In [ ]:
ast_df_grouped = ast_df.groupby('PLAYER_NAME').agg({'AST': np.sum })
highest_ast_idx = ast_df_grouped.AST.argmax()
ast_df_grouped.iloc[highest_ast_idx]

#### Solution (if I am not making a mistake somewhere): Chris Paul

### Exercise

Who is the most accurate shooter inside the 3 that has tried at least 1000 shots

Note: FGM stands for Field Goal Made, while FGA stands for Field Goal Attempted

In [ ]:
# Pseudocode - Step by step guide
# 1. Keep only columns of interest
# 2. Drop null FGA
# 3. Group by player
# 4. Filter df in order to keep only players with sum(FGA) > 1000
# 5. Create a new column with sum(FGM) / sum(FGA) as a value
# 6. Pick the index with the highest value in this division


In [ ]:
columns_to_keep_ex_3 = ['PLAYER_ID', 'PLAYER_NAME', 'FGM', 'FGA']
fgm_fga_df = df.drop([col for col in df.columns if col not in columns_to_keep_ex_3], axis=1)

In [ ]:
fgm_fga_df = fgm_fga_df[fgm_fga_df['FGA'].notna()]

In [ ]:
fgm_fga_df_grouped = fgm_fga_df.groupby('PLAYER_NAME').agg({'FGM': 'sum', 'FGA': 'sum'})

In [ ]:
more_than_1000_fga_mask = fgm_fga_df_grouped['FGA'] > 1000
fgm_fga_df_grouped = fgm_fga_df_grouped[more_than_1000_fga_mask]
fgm_fga_df_grouped['FG_PRECISION'] = fgm_fga_df_grouped['FGM'] / fgm_fga_df_grouped['FGA']

In [ ]:
highest_precision_position = fgm_fga_df_grouped.FG_PRECISION.argmax()
fgm_fga_df_grouped.iloc[highest_precision_position]

#### Solution (if I haven't made any mistake somewhere): Mitchell Robinson

### Exercise

Find common columns in teams.csv and games_details.csv

In [ ]:
teams_df_path = os.path.join(os.path.curdir, "sample_data", "teams.csv")
teams_df = pd.read_csv(teams_df_path)

games_details_path = os.path.join(os.path.curdir, "sample_data", "games_details.csv")
games_details_df = pd.read_csv(games_details_path)

teams_df_columns = teams_df.columns.tolist()
games_details_columns = games_details_df.columns.tolist()

starting_list = teams_df_columns if len(teams_df_columns) > len(games_details_columns) else games_details_columns
other_list = games_details_columns if len(teams_df_columns) > len(games_details_columns) else teams_df_columns

common_columns = [col for col in starting_list if col in other_list]
common_columns

#### Solution: TEAM_ID, NICKNAME

Note: there could be an alternative way to find this information, merging the two DataFrames and looking for columns ending with "_x" or "_y"

### Exercise

Merge the games_details.csv and teams.csv, then games_details with one of the other csvs

In [ ]:
# Pseudocode - Step by step guide
# 1. Load games_details.csv, teams.csv and games.cdv
# 2. Merge games_details.csv with teams.csv on TEAM_ID (inner)
# 3. Merge games_details.with games.csv on GAME_ID (inner)

In [ ]:
teams_df_path = os.path.join(os.path.curdir, "sample_data", "teams.csv")
teams_df = pd.read_csv(teams_df_path)

games_details_path = os.path.join(os.path.curdir, "sample_data", "games_details.csv")
games_details_df = pd.read_csv(games_details_path)

games_path = os.path.join(os.path.curdir, "sample_data", "games.csv")
games_df = pd.read_csv(games_path)

teams_games_details_df = pd.merge(teams_df, games_details_df, how="inner", left_on="TEAM_ID", right_on="TEAM_ID")
games_details_games_df = pd.merge(games_details_df, games_df, how="inner", left_on="GAME_ID", right_on="GAME_ID")

### Exercise

Find out who has the best PLUS_MINUS on average in Atlanta

In [ ]:
# Pseudocode - Step by step guide
# 1. Keep only subset of columns of interest
# 2. Drop null values
# 3. Filter out records that are not related to Atlanta
# 4. Group by player
# 5. Apply mean to PLUS_MINUS column values
# 6. Keep the highest one

In [ ]:
columns_to_keep_ex_4 = ['TEAM_CITY', 'PLAYER_NAME', 'PLUS_MINUS']
plus_minus_all_df = df.drop([col for col in df.columns if col not in columns_to_keep_ex_4], axis=1)

In [ ]:
plus_minus_atlanta_df = plus_minus_all_df[plus_minus_all_df.TEAM_CITY.str.contains('Atlanta', case=False)]

In [ ]:
plus_minus_atlanta_grouped_df = plus_minus_atlanta_df.groupby('PLAYER_NAME').aggregate({'PLUS_MINUS': 'mean'})

In [ ]:
highest_plus_minus_atlanta_loc = plus_minus_atlanta_grouped_df.PLUS_MINUS.argmax()
plus_minus_atlanta_grouped_df.iloc[highest_plus_minus_atlanta_loc]

#### Solution (if I am not mistaken): Paul Watson

### Exercise

Find who was the owner of LeBron James' team in 2016

In [ ]:
# Pseudocode - Step  by step guide
# Load Players.csv and Teams.csv
# Filter out players_df to have only record with PLAYER_NAME = "Lebron James" and SEASON = "2016"
# Merge the two dfs on TEAM_ID (inner)
# Look for OWNER column

In [ ]:
teams_df_path = os.path.join(os.path.curdir, "sample_data", "teams.csv")
teams_df = pd.read_csv(teams_df_path)

players_path = os.path.join(os.path.curdir, "sample_data", "players.csv")
players_df = pd.read_csv(players_path)

In [ ]:
players_df_filtered = players_df[(players_df.PLAYER_NAME.str.contains("Lebron James", case=False)) & (players_df.SEASON == 2016)]

In [ ]:
player_teams_df = pd.merge(players_df_filtered, teams_df, how="inner", left_on="TEAM_ID", right_on="TEAM_ID")

In [ ]:
player_teams_df['OWNER'].values[0]

#### Solution (if I am not mistaken): Dan Gilbert